<a href="https://colab.research.google.com/github/mrszkop/dw_matrix_road_sign/blob/master/Copy_of_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train= to_categorical(y_train)
if y_test.ndim == 1: y_test= to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1: ]
num_classes = y_train.shape[1]

In [0]:
def train_model(model,X_train,y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size',128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('vetbose',1),
      validation_data=params_fit.get('validation_data',(X_train,y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:

def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5(input_shape,num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
])


train_and_predict( get_cnn_v5(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 13s 49ms/step - loss: 2.4464 - accuracy: 0.3222 - val_loss: 0.7713 - val_accuracy: 0.7575
Epoch 2/5
272/272 [==============================] - 13s 47ms/step - loss: 0.6993 - accuracy: 0.7753 - val_loss: 0.1667 - val_accuracy: 0.9434
Epoch 3/5
272/272 [==============================] - 13s 47ms/step - loss: 0.3290 - accuracy: 0.8984 - val_loss: 0.0640 - val_accuracy: 0.9833
Epoch 4/5
272/272 [==============================] - 13s 47ms/step - loss: 0.2234 - accuracy: 0.9309 - val_loss: 0.0469 - val_accuracy: 0.9870
Epoch 5/5
272/272 [==============================] - 13s 47ms/step - loss: 0.1787 - accuracy: 0.9464 - val_loss: 0.0324 - val_accuracy: 0.9902


0.9673469387755103

In [0]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 48ms/step - loss: 2.3960 - accuracy: 0.3226 - val_loss: 0.9958 - val_accuracy: 0.7044
Epoch 2/5
272/272 [==============================] - 13s 47ms/step - loss: 0.7769 - accuracy: 0.7531 - val_loss: 0.1868 - val_accuracy: 0.9523
Epoch 3/5
272/272 [==============================] - 13s 47ms/step - loss: 0.3378 - accuracy: 0.8948 - val_loss: 0.0700 - val_accuracy: 0.9818
Epoch 4/5
272/272 [==============================] - 13s 47ms/step - loss: 0.2289 - accuracy: 0.9315 - val_loss: 0.0388 - val_accuracy: 0.9888
Epoch 5/5
272/272 [==============================] - 13s 47ms/step - loss: 0.1585 - accuracy: 0.9527 - val_loss: 0.0310 - val_accuracy: 0.9912


0.9727891156462585

In [0]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 7ms/step - loss: 0.0957 - accuracy: 0.9728


0.9727891087532043

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
])


train_and_predict( get_cnn_v5(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 15s 56ms/step - loss: 2.7793 - accuracy: 0.2398 - val_loss: 1.6704 - val_accuracy: 0.4361
Epoch 2/5
272/272 [==============================] - 13s 46ms/step - loss: 1.6057 - accuracy: 0.4656 - val_loss: 0.9738 - val_accuracy: 0.6850
Epoch 3/5
272/272 [==============================] - 13s 47ms/step - loss: 0.9145 - accuracy: 0.7018 - val_loss: 0.3357 - val_accuracy: 0.8975
Epoch 4/5
272/272 [==============================] - 13s 46ms/step - loss: 0.5277 - accuracy: 0.8299 - val_loss: 0.1604 - val_accuracy: 0.9507
Epoch 5/5
272/272 [==============================] - 13s 47ms/step - loss: 0.3629 - accuracy: 0.8877 - val_loss: 0.1414 - val_accuracy: 0.9522


0.9002267573696145

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size',128)),
      epochs=5,
      verbose=0,
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(params))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4984556677467039, 'dropout_cnn_block_three': 0.39592199918493337, 'dropout_cnn_block_two': 0.32648676761914674, 'dropout_dense_block_one': 0.30940917318975036, 'dropout_dense_block_two': 0.3652225894127789}
accuracy={'batch_size': 150.0, 'dropout_cnn_block_one': 0.4984556677467039, 'dropout_cnn_block_three': 0.39592199918493337, 'dropout_cnn_block_two': 0.32648676761914674, 'dropout_dense_block_one': 0.30940917318975036, 'dropout_dense_block_two': 0.3652225894127789}
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4093354190830977, 'dropout_cnn_block_three': 0.3984840651626991, 'dropout_cnn_block_two': 0.4056090636285412, 'dropout_dense_block_one': 0.5489539039741482, 'dropout_dense_block_two': 0.38063404334583634}
accuracy={'batch_size': 110.0, 'dropout_cnn_block_one': 0.4093354190830977, 'dropout_cnn_block_three': 0.3984840651626991, 'dropout_cnn_block_two': 0.4056090636285412, 'dropout_dense_block_one': 0.5489539039741482, 'dropout_d